In [1]:
# import csv
# file = open("WHO_FAQ.csv")
# csvreader = csv.reader(file)
# header = next(csvreader)
# print(header)
# rows = []
# for row in csvreader:
#     rows.append(row)
# print(rows)
# file.close()

In [2]:
# for i in rows:
#   print('{"pats":["'+str(i[0])+'"],\n"replies":["'+str(i[1]) +'"]},\n')

In [154]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [155]:
import json 
import numpy as np 
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [166]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [167]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [168]:
punc_list=['.','/','"','-','?',',',"'"]
covid_vars=["COVID", "Corona", "Coronavirus", "COVID-19"]
exclude_list=["not","nor","don't","what","how","are","you"]
stop_words = [i for i in stopwords.words('english') if (i not in exclude_list)]

def preprocess(example_sent):
  example_sent=example_sent.replace("covid-19","covid").replace("coronavirus","covid").replace("corona","covid").replace("covid19","covid").replace("virus","covid")
  word_tokens = word_tokenize(example_sent)
  filtered_sentence = [stemmer.stem(w) for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)
  for i in punc_list:
    if i in filtered_sentence:
      filtered_sentence.remove(i)
  return ' '.join(filtered_sentence)

In [169]:
with open('intents.json') as file:
    data = json.load(file)
    
train_x = []
train_y = []
bag = []
replies = []


for intent in data['intents']:
    for pat in intent['patterns']:
        s=preprocess(pat)
        train_x.append(s.lower())
        print(s.lower())
        train_y.append(intent['tag'].lower())
    replies.append([i.lower() for i in intent['responses']])
    
    if intent['tag'] not in bag:
        bag.append(intent['tag'].lower())
        
num_classes = len(bag)

hi
hey
is anyone
hello
hay
sup
how are you
how are you
are you fine
how you
how life
how you
bye
see you later
goodbye
thanks
thank you
that 's helpful
thanks help
okay
ok
who are you
what are you
who
what name
what i call you
whats name
your name
could you help
give hand please
can you help
what you
i need support
i need help
support please
what covid
what covid
what covid
what covid
what pandemic
what pandemic
what covid
what covid
what covid
what covid
what covid
what are symptoms covid
symptoms covid
what happens covid
what happens you get covid
what are symptoms covid
get fever covid
what happens i get covid
what get covid
what are symptoms covid
what are symptoms covid
fever symptom covid
cough symptom covid
do old people get covid faster
how covid rise
how covid increase
how people catch covid
how peole get covid
how covid spread
how covid spread
covid spread
how people catch covid
how covid spread
can covid spread air
can covid move air
covid spread air
can covid spread coughin

In [170]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(train_y)
train_y = lbl_encoder.transform(train_y)

In [171]:
max_len = 50
oov_token = "<OOV>"
vocab_size = 10000
embedding_dim = 16

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_x)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_x)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [172]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 50, 16)            160000    
                                                                 
 global_average_pooling1d_8   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_30 (Dense)            (None, 16)                272       
                                                                 
 dense_31 (Dense)            (None, 16)                272       
                                                                 
 dense_32 (Dense)            (None, 40)                680       
                                                                 
Total params: 161,224
Trainable params: 161,224
Non-trainable params: 0
_______________________________________________

In [173]:
print(train_y)

[12 12 12 12 12 12  4  4  4  4  4  4  7  7  7 36 36 36 36 36 36  0  0  0
 18 18 18 18 13 13 13 13 13 13 13  9  9  9  9  9  9  9  9  9  9  9 35 35
 35 35 35 35 35 35 35 20 20 20 20 33 33 33 33 33 33 33 33 33 33 33 33 33
 33 33 33 33 33 33 33 33 35 35 35 35 35 35 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 29 29 29 29 29 29 29  8  8  8  8  8  8  8  8 27 27 27 27 27 27
 27 27 27 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
 28 28 28 28 28 28 28 28 28 28  3  3  3  3  3  3  3  3  3  3  3 17 17 17
 17 17 17 17 17 17 17 37 37 37 37 37 17 17 17 17 17 17 17 17 17 30 30 30
 30 30 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 25 25 25 25 25 25 25  2  2  2  2  2  2  2  2  2  2  2  2  2
 22 22 22 22 22 22 22 34 34 34 34 34 34 34 34 34 34 21 21 21 21 21 21 19
 19 19 19 19 19 19 32 32 32 32 32 10 10 10 10 10 10 10 10  1  1  1  1  1
  1  1  1 23 23 23 23 23 23 23 23 23 23 23 23 23 23

In [174]:
epochs = 750
history = model.fit(padded_sequences, np.array(train_y), epochs=epochs)

Epoch 1/750
13/13 [==============================] - 1s 16ms/step - loss: 3.6877 - accuracy: 0.0662
Epoch 2/750
13/13 [==============================] - 0s 13ms/step - loss: 3.6818 - accuracy: 0.0941
Epoch 3/750
13/13 [==============================] - 0s 12ms/step - loss: 3.6767 - accuracy: 0.0941
Epoch 4/750
13/13 [==============================] - 0s 13ms/step - loss: 3.6713 - accuracy: 0.0941
Epoch 5/750
13/13 [==============================] - 0s 12ms/step - loss: 3.6655 - accuracy: 0.0941
Epoch 6/750
13/13 [==============================] - 0s 8ms/step - loss: 3.6588 - accuracy: 0.0941
Epoch 7/750
13/13 [==============================] - 0s 15ms/step - loss: 3.6512 - accuracy: 0.0916
Epoch 8/750
13/13 [==============================] - 0s 11ms/step - loss: 3.6408 - accuracy: 0.0789
Epoch 9/750
13/13 [==============================] - 0s 11ms/step - loss: 3.6257 - accuracy: 0.0789
Epoch 10/750
13/13 [==============================] - 0s 13ms/step - loss: 3.6066 - accuracy: 0.0789


In [179]:
model.save("final_covid_chatbot_model")


import pickle

# to save the fitted tokenizer
with open('tokenizer_final.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder_final.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: final_covid_chatbot_model/assets


INFO:tensorflow:Assets written to: final_covid_chatbot_model/assets


  adding: covid_chatbot_model9/ (stored 0%)
  adding: covid_chatbot_model9/variables/ (stored 0%)
  adding: covid_chatbot_model9/variables/variables.index (deflated 62%)
  adding: covid_chatbot_model9/variables/variables.data-00000-of-00001 (deflated 48%)
  adding: covid_chatbot_model9/assets/ (stored 0%)
  adding: covid_chatbot_model9/saved_model.pb (deflated 90%)
  adding: covid_chatbot_model9/keras_metadata.pb (deflated 88%)


In [180]:
!zip -r final_chatmodel.zip final_covid_chatbot_model

  adding: final_covid_chatbot_model/ (stored 0%)
  adding: final_covid_chatbot_model/variables/ (stored 0%)
  adding: final_covid_chatbot_model/variables/variables.index (deflated 64%)
  adding: final_covid_chatbot_model/variables/variables.data-00000-of-00001 (deflated 67%)
  adding: final_covid_chatbot_model/assets/ (stored 0%)
  adding: final_covid_chatbot_model/saved_model.pb (deflated 88%)
  adding: final_covid_chatbot_model/keras_metadata.pb (deflated 88%)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [114]:
!pip install colorama

In [175]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    # model = keras.models.load_model('covid_chatbot_model9')

    # # load tokenizer object
    # with open('tokenizer9.pickle', 'rb') as handle:
    #     tokenizer = pickle.load(handle)

    # # load label encoder object
    # with open('label_encoder9.pickle', 'rb') as enc:
    #     lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 50
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        s=preprocess(inp)
        inp=s.lower()
        print(tokenizer.texts_to_sequences([inp]))
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag[0].lower():
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(replies))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
[[182]]
ChatBot: Hi there
User: how are you
[[3, 5, 14]]
ChatBot: Im great!
User: what is covid
[[4, 2]]
ChatBot: Coronaviruses are a large family of viruses which may cause illness in animals or humans. 
ChatBot: covid is a virus that caused the current pandemic
ChatBot: Avoid stress and stay vaccinated
User: how to increase immunity?
[[3, 90, 63]]
ChatBot: In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases
ChatBot: covid is a virus that caused the current pandemic
ChatBot: Stay physically active. and get sleep
User: bye
[[121]]
ChatBot: Have a nice day
ChatBot: Great! See you later!
User: quit
[[1]]


In [176]:
corr=0
tot=0
for x in data['intents']:
  for inp in x['patterns']:
    tot+=1
    s=preprocess(inp)
    inp=s.lower()

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                          truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])
    if tag[0].lower()==x['tag'].lower():
      corr+=1

In [177]:
print("Correct: {}\nTotal: {}".format(corr,tot))

Correct: 370
Total: 393


In [178]:
print("Accuracy: {}".format(corr*100/tot))

Accuracy: 94.14758269720102
